In [191]:
%matplotlib inline
import pandas as pd
import numpy as np
import json
import xml.etree.ElementTree as et
import nltk
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.corpus import stopwords
from nltk.collocations import *
import os

In [99]:
import sys  
reload(sys)  
sys.setdefaultencoding('utf8')

In [101]:
df_events.shape

(77257, 31)

In [102]:
nar_list = []
filelist = [name for name in os.listdir("../data/") if name.endswith(".json")]
for f in filelist:
    with open(os.path.join("../data/", f)) as json_file:
        j = json.load(json_file)['data']
        nar_list += j
df_nar = pd.DataFrame(nar_list)

In [103]:
df_combined = pd.merge(df_events, df_nar,how='left',on='EventId')

In [104]:
df_pc = df_combined[(df_combined['probable_cause']!='')]

In [105]:
df_pc.head()

,AccidentNumber,AirCarrier,AircraftCategory,AircraftDamage,AirportCode,AirportName,AmateurBuilt,BroadPhaseOfFlight,Country,EngineType,...,RegistrationNumber,ReportStatus,Schedule,TotalFatalInjuries,TotalMinorInjuries,TotalSeriousInjuries,TotalUninjured,WeatherCondition,narrative,probable_cause
175,GAA15CA208,,Airplane,Substantial,OVO,NORTH VERNON,No,LANDING,United States,Reciprocating,...,N133VU,Probable Cause,,,,,1,VMC,NTSB investigators used data provided by vario...,The pilot's failure to maintain pitch control ...
183,GAA15CA197,,Helicopter,Substantial,,N/A,No,LANDING,United States,Reciprocating,...,N58397,Probable Cause,,,,,2,VMC,NTSB investigators used data provided by vario...,The non-helicopter rated pilot's improper land...
203,GAA15CA189,,Airplane,Substantial,U660,Hidden Splendor,No,TAKEOFF,United States,Reciprocating,...,N735JB,Probable Cause,,,,,2,VMC,NTSB investigators used data provided by vario...,The pilot's failure to maintain directional co...
205,ERA15CA272,,Airplane,Substantial,DVK,Stuart Powell Field,Yes,GO-AROUND,United States,Reciprocating,...,N31CS,Probable Cause,,,1,,,VMC,NTSB investigators used data provided by vario...,"The pilot's inaccurate fuel planning, which re..."
217,WPR15CA214,,Airplane,Substantial,K69S,Avey Field,No,LANDING,United States,Reciprocating,...,C-GDYN,Probable Cause,,,1,,,VMC,NTSB investigators used data provided by vario...,The pilot's inadequate decision to continue th...


## Start Text Analysis

In [173]:
pc_list = df_pc['probable_cause'].tolist()
pc_list = [i for i in pc_list]
pc_string = str(''.join(pc_list))
pc_string = unicode(pc_string, errors='replace')

In [177]:
pc_string = pc_string.replace(u"\u2018", "'").replace(u"\u2019", "'")

In [192]:
sw = stopwords.words('english')
tokens = nltk.word_tokenize(pc_string)
text = nltk.Text(tokens)
words = [w.lower() for w in text if w.isalpha()]
words = [w for w in words if w not in sw]

In [206]:
fd = nltk.FreqDist(words)
bi_fd = nltk.FreqDist(nltk.bigrams(words))

In [214]:
fd.most_common(30)

[(u'pilot', 41897),
 (u'failure', 28745),
 (u'landing', 17154),
 (u'maintain', 15560),
 (u'control', 12881),
 (u'loss', 12856),
 (u'resulted', 12614),
 (u'accident', 12037),
 (u'engine', 11589),
 (u'factor', 10112),
 (u'power', 9648),
 (u'fuel', 9482),
 (u'inadequate', 9131),
 (u'factors', 9069),
 (u'contributing', 8705),
 (u'flight', 8367),
 (u'airplane', 8048),
 (u'improper', 7994),
 (u'due', 7596),
 (u'terrain', 7433),
 (u'conditions', 6257),
 (u'aircraft', 5752),
 (u'directional', 5461),
 (u'resulting', 4973),
 (u'lack', 4789),
 (u'takeoff', 4635),
 (u'runway', 4552),
 (u'undetermined', 4339),
 (u'airspeed', 4004),
 (u'subsequent', 3827)]

In [198]:
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [207]:
finder = BigramCollocationFinder(fd, bi_fd)

In [213]:
sorted(finder.nbest(bigram_measures.raw_freq, 30))

[(u'accident', u'pilot'),
 (u'aircraft', u'control'),
 (u'contributing', u'accident'),
 (u'contributing', u'factor'),
 (u'contributing', u'factors'),
 (u'control', u'airplane'),
 (u'control', u'landing'),
 (u'directional', u'control'),
 (u'engine', u'power'),
 (u'factor', u'accident'),
 (u'failure', u'maintain'),
 (u'failure', u'pilot'),
 (u'forced', u'landing'),
 (u'fuel', u'exhaustion'),
 (u'inadequate', u'preflight'),
 (u'landing', u'gear'),
 (u'loss', u'control'),
 (u'loss', u'engine'),
 (u'maintain', u'adequate'),
 (u'maintain', u'clearance'),
 (u'maintain', u'directional'),
 (u'pilot', u'failure'),
 (u'pilot', u'improper'),
 (u'pilot', u'inadequate'),
 (u'pilot', u'maintain'),
 (u'power', u'due'),
 (u'power', u'undetermined'),
 (u'resulted', u'loss'),
 (u'student', u'pilot'),
 (u'undetermined', u'reasons')]

While some of the bigram above doesn't exactly make sense grammatically, this happens because of the removal of stopwords. (aircraft,control) originally was most likely to be "aircraft out of control".

In [186]:
def ispilotfailure(x):
    if ('pilot' in x.lower()) & ('failure' in x.lower()):
        return 1

In [188]:
df_pc['pc'] = df_pc['probable_cause'].apply(ispilotfailure)

In [190]:
df_pc.head()

,AccidentNumber,AirCarrier,AircraftCategory,AircraftDamage,AirportCode,AirportName,AmateurBuilt,BroadPhaseOfFlight,Country,EngineType,...,ReportStatus,Schedule,TotalFatalInjuries,TotalMinorInjuries,TotalSeriousInjuries,TotalUninjured,WeatherCondition,narrative,probable_cause,pc
175,GAA15CA208,,Airplane,Substantial,OVO,NORTH VERNON,No,LANDING,United States,Reciprocating,...,Probable Cause,,,,,1,VMC,NTSB investigators used data provided by vario...,The pilot's failure to maintain pitch control ...,1.0
183,GAA15CA197,,Helicopter,Substantial,,N/A,No,LANDING,United States,Reciprocating,...,Probable Cause,,,,,2,VMC,NTSB investigators used data provided by vario...,The non-helicopter rated pilot's improper land...,NaN
203,GAA15CA189,,Airplane,Substantial,U660,Hidden Splendor,No,TAKEOFF,United States,Reciprocating,...,Probable Cause,,,,,2,VMC,NTSB investigators used data provided by vario...,The pilot's failure to maintain directional co...,1.0
205,ERA15CA272,,Airplane,Substantial,DVK,Stuart Powell Field,Yes,GO-AROUND,United States,Reciprocating,...,Probable Cause,,,1,,,VMC,NTSB investigators used data provided by vario...,"The pilot's inaccurate fuel planning, which re...",NaN
217,WPR15CA214,,Airplane,Substantial,K69S,Avey Field,No,LANDING,United States,Reciprocating,...,Probable Cause,,,1,,,VMC,NTSB investigators used data provided by vario...,The pilot's inadequate decision to continue th...,NaN


In [3]:
text = 'The failure of the compressor section No. 2 bearing due to false brinnelling and fatigue. This is the next sentence.'

In [4]:
sents = sent_tokenize(text)

In [5]:
sents

['The failure of the compressor section No.',
 '2 bearing due to false brinnelling and fatigue.',
 'This is the next sentence.']